In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
url = 'https://liquipedia.net/valorant/VCT/2024/Stage_1/Masters/Statistics'

file = requests.get(url)

In [3]:
print(file.text)

<!DOCTYPE html>
<html class="client-nojs Send_pizza_to_FO-nTTaX All_glory_to_Liquipedia theme--light" lang="en" dir="ltr" prefix="og: http://ogp.me/ns#">
<head>
<meta charset="UTF-8"/>
<title>VALORANT Champions Tour 2024: Masters Madrid - Statistics - Liquipedia VALORANT Wiki</title>
<script>document.documentElement.className="client-js Send_pizza_to_FO-nTTaX All_glory_to_Liquipedia theme--light";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"e7dfadb2d141342355af555f","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"VCT/2024/Stage_1/Masters/Statistics","wgTitle":"VCT/2024/Stage 1/Masters/Statistics","wgCurRevisionId":870385,"wgRevisionId":870385,"wgArticleId":52719,"wgIsArticle":true,"wgIsRedirect":f

In [4]:
soup = BeautifulSoup(file.text, 'html.parser')


soup.select('table')

[<table class="wikitable wikitable-striped sortable" style="margin-top:10px;text-align:center"><tbody><tr><th width="10">#</th><th width="200">Player</th><th width="50">Team</th><th width="75">Agents</th><th width="50">Maps</th><th width="50">K</th><th width="50">D</th><th width="50">A</th><th width="50">KD</th><th width="50">KDA</th><th width="50">ACS/Map</th><th width="50">K/Map</th><th width="50">D/Map</th><th width="50">A/Map</th></tr><tr><td>1</td><td style="text-align:left"><span class="flag"><a href="/valorant/Category:United_States" title="United States"><img alt="United States" decoding="async" height="24" src="/commons/images/thumb/3/3b/Us_hd.png/36px-Us_hd.png" srcset="/commons/images/thumb/3/3b/Us_hd.png/54px-Us_hd.png 1.5x, /commons/images/thumb/3/3b/Us_hd.png/72px-Us_hd.png 2x" width="36"/></a></span> <a href="/valorant/Zekken" title="Zekken">zekken</a></td><td><span class="team-template-team-icon" data-highlightingclass="Sentinels"><span class="team-template-image-icon t

In [5]:
table1 = soup.find('table', attrs={'class' :'wikitable wikitable-striped sortable'})


table1_data = table1.tbody.find_all('tr')

In [19]:
print(type(table1))

<class 'bs4.element.Tag'>


In [6]:

headings=[]

for td in table1_data[0].find_all('th'):

    
    headings.append(td.text)
print(headings)

['#', 'Player', 'Team', 'Agents', 'Maps', 'K', 'D', 'A', 'KD', 'KDA', 'ACS/Map', 'K/Map', 'D/Map', 'A/Map']


In [7]:
#created a dictionary to store player demographics

demo={'Player':[],'Country':[],'Team':[],'Agents':[]}
final_stat = []

# looping through the rows starting from the second row
for i in range(1,len(table1_data)):
    temp = [] # temporary list for player demographics
    stat_temp = [] # temportary list for player stats

        
    # Loop through the cells in the current row then append the stats into the stat_temp file
    for td in table1_data[i].find_all('td'):
        stat_temp.append(td.text) 

                
        # Loop through the <a> element within the cell we are adding the title to the demo temp
        for t in td.find_all('a'):
            temp.append(t.get('title'))

    # using the dictionary we created earlier we extract and store players demographics to 
    # correspoinding column

    # although country doesn't have its own column the images have names corresponding to the country

    demo['Country'].append(temp[0])
    demo['Player'].append(temp[1])
    demo['Team'].append(temp[2])
    demo['Agents'].append(temp[4:])
    
    #i then append the players stats to the stat_temp file
    final_stat.append(stat_temp)

# creating data frames based on the players demographic and stats
info=pd.DataFrame(demo)
stat=pd.DataFrame(final_stat)

# since we didn't remove anything from the stat data frame we would need to drop the first 
# 4 columns as they are the demographics we seperated
stat.drop([0,1,2,3],axis=1,inplace=True)

# we then rename the columns using the headings list we created earlier on
stat.columns=headings[4:]

In [9]:
stat.head()

,Maps,K,D,A,KD,KDA,ACS/Map,K/Map,D/Map,A/Map
0,20,381,314,90,1.21,1.5,259,19,15.7,4.5
1,8,135,125,39,1.08,1.39,252,16.8,15.6,4.8
2,7,132,104,65,1.26,1.89,245,18.8,14.8,9.2
3,6,114,92,37,1.23,1.64,245,19,15.3,6.1
4,4,68,71,16,0.95,1.18,231,17,17.7,4


In [10]:
info.head()

,Player,Country,Team,Agents
0,Zekken,United States,Sentinels,"[Raze, Jett]"
1,ZmjjKK,China,EDward Gaming,"[Raze, Jett, KAY/O]"
2,N4RRATE,United States,Karmine Corp,"[Fade, Raze, Gekko]"
3,MiniBoo,Lithuania,Team Heretics,"[Neon, Raze, Yoru]"
4,Autumn (Australian player),Australia,FunPlus Phoenix,"[Jett, Raze, Phoenix]"


In [11]:
Player_Statistics = pd.merge(info, stat, left_index=True, right_index=True)

In [12]:
Player_Statistics.head()

,Player,Country,Team,Agents,Maps,K,D,A,KD,KDA,ACS/Map,K/Map,D/Map,A/Map
0,Zekken,United States,Sentinels,"[Raze, Jett]",20,381,314,90,1.21,1.5,259,19,15.7,4.5
1,ZmjjKK,China,EDward Gaming,"[Raze, Jett, KAY/O]",8,135,125,39,1.08,1.39,252,16.8,15.6,4.8
2,N4RRATE,United States,Karmine Corp,"[Fade, Raze, Gekko]",7,132,104,65,1.26,1.89,245,18.8,14.8,9.2
3,MiniBoo,Lithuania,Team Heretics,"[Neon, Raze, Yoru]",6,114,92,37,1.23,1.64,245,19,15.3,6.1
4,Autumn (Australian player),Australia,FunPlus Phoenix,"[Jett, Raze, Phoenix]",4,68,71,16,0.95,1.18,231,17,17.7,4


In [30]:
# I then selected the first table which according to the website is their player statistics
table2 = soup.find('table', class_= 'wikitable wikitable-bordered wikitable-striped')

# I then extrated all the data in said row


In [31]:
print(type(table2))

<class 'bs4.element.Tag'>


In [32]:
table2_data = table2.tbody.find_all('tr')

In [42]:
final_data2 = []

# looping through the rows starting from the first row
for i in range(len(table2_data)):
    temp = [] # temporary list for player demographics
       
    # Loop through the cells in the current row then append the stats into the temp file
    for td in table2_data[i].find_all(['td','th']):
        temp.append(td.text) 
    
    # I then append the data to the temp file
    final_data2.append(temp)

# creating data frames based on the map pick
map_pick=pd.DataFrame(final_data2)

# Set the first row as column names
map_pick.columns = map_pick.iloc[0]

# Drop the first row 
map_pick.drop(0,inplace=True)

# Set the 'Map' column as the index
map_pick.set_index(keys='Map', inplace=True)

In [43]:
map_pick

,Total,Round 1,Round 2,Round 3,Playoffs
Map,,,,,
Ascent,6×,1×,0×,1×,4×
Bind,7×,2×,1×,1×,3×
Breeze,6×,1×,2×,1×,2×
Icebox,7×,2×,2×,1×,2×
Lotus,8×,2×,2×,1×,3×
Split,9×,1×,3×,0×,5×
Sunset,9×,3×,2×,1×,3×


In [44]:
for i in map_pick.columns:
    map_pick[i]=map_pick[i].apply(lambda x:int(x[:-1]))

In [45]:
map_pick

,Total,Round 1,Round 2,Round 3,Playoffs
Map,,,,,
Ascent,6,1,0,1,4
Bind,7,2,1,1,3
Breeze,6,1,2,1,2
Icebox,7,2,2,1,2
Lotus,8,2,2,1,3
Split,9,1,3,0,5
Sunset,9,3,2,1,3


In [46]:
Player_Statistics.to_csv('Data - Player Statistics.csv',index=False)

In [47]:
map_pick.to_csv('Data - Map Picked.csv')

In [54]:
html_content = '''
<table class="wikitable wikitable-bordered wikitable-striped"><tbody><tr><th>Map</th><th><abbr title="Attack">Atk</abbr> Wins</th><th><abbr title="Defense">Def</abbr> Wins</th></tr><tr><td><b><a href="/valorant/Ascent" title="Ascent">Ascent</a></b></td><td>54 (<i>50.0%</i>)</td><td>54 (<i>50.0%</i>)</td></tr><tr><td><b><a href="/valorant/Bind" title="Bind">Bind</a></b></td><td>47 (<i>49.5%</i>)</td><td>48 (<i>50.5%</i>)</td></tr><tr><td><b><a href="/valorant/Breeze" title="Breeze">Breeze</a></b></td><td>56 (<i>55.4%</i>)</td><td>45 (<i>44.6%</i>)</td></tr><tr><td><b><a href="/valorant/Icebox" title="Icebox">Icebox</a></b></td><td>68 (<i>54.0%</i>)</td><td>58 (<i>46.0%</i>)</td></tr><tr><td><b><a href="/valorant/Lotus" title="Lotus">Lotus</a></b></td><td>98 (<i>54.7%</i>)</td><td>81 (<i>45.3%</i>)</td></tr><tr><td><b><a href="/valorant/Split" title="Split">Split</a></b></td><td>80 (<i>44.9%</i>)</td><td>98 (<i>55.1%</i>)</td></tr><tr><td><b><a href="/valorant/Sunset" title="Sunset">Sunset</a></b></td><td>82 (<i>51.6%</i>)</td><td>77 (<i>48.4%</i>)</td></tr></tbody></table>
'''

# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find the table element
table3 = soup.find('table', class_='wikitable')

# Initialize lists to store data
maps = []
atk_wins = []
def_wins = []

# Extract data from each row of the table
for row in table3.find_all('tr')[1:]:
    cols = row.find_all('td')
    maps.append(cols[0].text.strip())
    atk_wins.append(cols[1].text.strip())
    def_wins.append(cols[2].text.strip())

# Create a pandas DataFrame
data = {'Map': maps, 'Attack Wins': atk_wins, 'Defense Wins': def_wins}
side_stats = pd.DataFrame(data)


In [55]:
side_stats

,Map,Attack Wins,Defense Wins
0,Ascent,54 (50.0%),54 (50.0%)
1,Bind,47 (49.5%),48 (50.5%)
2,Breeze,56 (55.4%),45 (44.6%)
3,Icebox,68 (54.0%),58 (46.0%)
4,Lotus,98 (54.7%),81 (45.3%)
5,Split,80 (44.9%),98 (55.1%)
6,Sunset,82 (51.6%),77 (48.4%)


In [58]:
side_stats['Attack Wins']=side_stats['Attack Wins'].apply(lambda x:int(x.split()[0]))
side_stats['Defense Wins']=side_stats['Defense Wins'].apply(lambda x:int(x.split()[0]))

In [59]:
side_stats

,Map,Attack Wins,Defense Wins
0,Ascent,54,54
1,Bind,47,48
2,Breeze,56,45
3,Icebox,68,58
4,Lotus,98,81
5,Split,80,98
6,Sunset,82,77


In [61]:
side_stats.to_csv('Data - Side Stats.csv', index=False)

In [64]:



html_content2 = '''
<table class="wikitable wikitable-bordered wikitable-striped"><tbody><tr><th style="text-align:left">Map</th><th style="text-align:center">Total</th><th style="text-align:center">Round 1</th><th style="text-align:center">Round 2</th><th style="text-align:center">Round 3</th><th style="text-align:center">Playoffs</th></tr><tr><th style="text-align:left"><b><a href="/valorant/index.php?title=-&amp;action=edit&amp;redlink=1" class="new" title="- (page does not exist)">-</a></b></th><th style="text-align:center">2<i>×</i></th><td style="text-align:center">0<i>×</i></td><td style="text-align:center">0<i>×</i></td><td style="text-align:center">0<i>×</i></td><td style="text-align:center">2<i>×</i></td></tr><tr><th style="text-align:left"><b><a href="/valorant/Ascent" title="Ascent">Ascent</a></b></th><th style="text-align:center">10<i>×</i></th><td style="text-align:center">3<i>×</i></td><td style="text-align:center">4<i>×</i></td><td style="text-align:center">1<i>×</i></td><td style="text-align:center">2<i>×</i></td></tr><tr><th style="text-align:left"><b><a href="/valorant/Bind" title="Bind">Bind</a></b></th><th style="text-align:center">9<i>×</i></th><td style="text-align:center">2<i>×</i></td><td style="text-align:center">3<i>×</i></td><td style="text-align:center">1<i>×</i></td><td style="text-align:center">3<i>×</i></td></tr><tr><th style="text-align:left"><b><a href="/valorant/Breeze" title="Breeze">Breeze</a></b></th><th style="text-align:center">10<i>×</i></th><td style="text-align:center">3<i>×</i></td><td style="text-align:center">2<i>×</i></td><td style="text-align:center">1<i>×</i></td><td style="text-align:center">4<i>×</i></td></tr><tr><th style="text-align:left"><b><a href="/valorant/Icebox" title="Icebox">Icebox</a></b></th><th style="text-align:center">9<i>×</i></th><td style="text-align:center">2<i>×</i></td><td style="text-align:center">2<i>×</i></td><td style="text-align:center">1<i>×</i></td><td style="text-align:center">4<i>×</i></td></tr><tr><th style="text-align:left"><b><a href="/valorant/Lotus" title="Lotus">Lotus</a></b></th><th style="text-align:center">8<i>×</i></th><td style="text-align:center">2<i>×</i></td><td style="text-align:center">2<i>×</i></td><td style="text-align:center">1<i>×</i></td><td style="text-align:center">3<i>×</i></td></tr><tr><th style="text-align:left"><b><a href="/valorant/Split" title="Split">Split</a></b></th><th style="text-align:center">7<i>×</i></th><td style="text-align:center">3<i>×</i></td><td style="text-align:center">1<i>×</i></td><td style="text-align:center">2<i>×</i></td><td style="text-align:center">1<i>×</i></td></tr><tr><th style="text-align:left"><b><a href="/valorant/Sunset" title="Sunset">Sunset</a></b></th><th style="text-align:center">7<i>×</i></th><td style="text-align:center">1<i>×</i></td><td style="text-align:center">2<i>×</i></td><td style="text-align:center">1<i>×</i></td><td style="text-align:center">3<i>×</i></td></tr></tbody></table>
'''

# Parse the HTML content
soup = BeautifulSoup(html_content2, 'html.parser')

# Find the table element
table = soup.find('table', class_='wikitable')

# Initialize lists to store data
maps = []
total = []
round_1 = []
round_2 = []
round_3 = []
playoffs = []

# Extract data from each row of the table
for row in table.find_all('tr')[1:]:
    cols = row.find_all(['th', 'td'])
    maps.append(cols[0].text.strip())
    total.append(cols[1].text.strip())
    round_1.append(cols[2].text.strip())
    round_2.append(cols[3].text.strip())
    round_3.append(cols[4].text.strip())
    if len(cols) > 5:  # Check if 'Playoffs' data exists
        playoffs.append(cols[5].text.strip())
    else:
        playoffs.append('')

# Create a pandas DataFrame
data = {'Map': maps, 'Total': total, 'Round 1': round_1, 'Round 2': round_2, 'Round 3': round_3, 'Playoffs': playoffs}
maps_ban = pd.DataFrame(data)



In [65]:
maps_ban

,Map,Total,Round 1,Round 2,Round 3,Playoffs
0,-,2×,0×,0×,0×,2×
1,Ascent,10×,3×,4×,1×,2×
2,Bind,9×,2×,3×,1×,3×
3,Breeze,10×,3×,2×,1×,4×
4,Icebox,9×,2×,2×,1×,4×
5,Lotus,8×,2×,2×,1×,3×
6,Split,7×,3×,1×,2×,1×
7,Sunset,7×,1×,2×,1×,3×


In [67]:
maps_ban.replace('×', '', regex=True, inplace=True)

In [68]:
maps_ban

,Map,Total,Round 1,Round 2,Round 3,Playoffs
0,-,2,0,0,0,2
1,Ascent,10,3,4,1,2
2,Bind,9,2,3,1,3
3,Breeze,10,3,2,1,4
4,Icebox,9,2,2,1,4
5,Lotus,8,2,2,1,3
6,Split,7,3,1,2,1
7,Sunset,7,1,2,1,3


In [69]:
maps_ban.drop(0,inplace=True)


In [70]:
maps_ban

,Map,Total,Round 1,Round 2,Round 3,Playoffs
1,Ascent,10,3,4,1,2
2,Bind,9,2,3,1,3
3,Breeze,10,3,2,1,4
4,Icebox,9,2,2,1,4
5,Lotus,8,2,2,1,3
6,Split,7,3,1,2,1
7,Sunset,7,1,2,1,3


In [71]:
maps_ban = maps_ban.reset_index(drop=True)

In [72]:
maps_ban

,Map,Total,Round 1,Round 2,Round 3,Playoffs
0,Ascent,10,3,4,1,2
1,Bind,9,2,3,1,3
2,Breeze,10,3,2,1,4
3,Icebox,9,2,2,1,4
4,Lotus,8,2,2,1,3
5,Split,7,3,1,2,1
6,Sunset,7,1,2,1,3


In [73]:
maps_ban.to_csv('Data - Map Ban.csv', index= False)